<a href="https://colab.research.google.com/github/zhe0/prac/blob/main/untitle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('在在colab存ipynb建議用github副本，確認有效．只是不能隨意按下隨意按下ctrl+s，仰賴檔案的存副本到存副本到github')

在在colab存ipynb建議用github副本，確認有效．只是不能隨意按下隨意按下ctrl+s，仰賴檔案的存副本到存副本到github


In [1]:
!apt-get install r-base r-base-dev
# !apt-get update
# !apt-get install --yes git ssh python3-venv
# !apt-get install r-base r-base-dev gdal-bin python-gdal python3-gdal libgdal-dev libproj-dev proj-data proj-bin libgeos-dev libudunits2-dev libv8-dev libprotobuf-dev libxml2 libjq-dev
# !apt-get install qgis saga

Reading package lists... Done
Building dependency tree       
Reading state information... Done
r-base-dev is already the newest version (4.2.2.20221110-1.2004.0).
r-base is already the newest version (4.2.2.20221110-1.2004.0).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [2]:
# !pip install rpy2
!pip install -Iv rpy2==3.4.2 

Using pip 22.0.4 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 KB 3.9 MB/s eta 0:00:00
  Running command python setup.py egg_info
  ['cffi>=1.10.0', 'jinja2', 'pytz', 'tzlocal']
  running egg_info
  creating /tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info
  writing /tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info/SOURCES.txt'
  package init file './rpy2/tests/rlike/__init__.py' not found (or not a regular file)
  reading manifest file '/tmp/pip-pip-egg-info-tg2ysngj/rpy2.egg-info/SOUR

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import globalenv

In [7]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [5]:
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2

In [7]:
import rpy2
import pandas as pd
df_py = pd.DataFrame({'col1':[1,2,3,4,5],
                      'col2':[1.0,2.0,3.0,4.0,5.0],
                      'col3':['a','b','c','d','e']})

from rpy2.robjects.pandas2ri import py2rpy, rpy2py
rpy2.robjects.pandas2ri.activate()

df_r = py2rpy(df_py)    
rpy2py(df_r)

,col1,col2,col3
0,1,1.0,a
1,2,2.0,b
2,3,3.0,c
3,4,4.0,d
4,5,5.0,e


In [3]:
# base = importr('base')
# del rpy2
import rpy2
print(rpy2.__version__)

3.4.2


In [ ]:
import numpy as np
np.set_printoptions(precision=4)
import pandas as pd
import pickle, os, gc, datetime, gc, re
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

from transformers import AutoModel
from transformers import (
   BertTokenizerFast,
#    AutoModelForMaskedLM,
#    AutoModelForTokenClassification,
#     AutoModelForSequenceClassification,
#     FlaxAutoModelForSequenceClassification
)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset,Dataset
from torch.utils.data.dataset import random_split
import torch.utils.data as data
# import torch_optimizer

# from u import *
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.has_mps else 'cpu'
torch.backends.cudnn.benchmark = True
import multiprocessing as mp
print(torch.get_num_threads())
torch.set_num_threads(mp.cpu_count()-1)
print(torch.get_num_threads(),device,mp.cpu_count())

7
7 mps 8


In [ ]:
torch.has_mps

True

In [ ]:
# pkl(path='raw.pkl',wbORrb='wb',datas=(raw,op,op_mid,op_big,op_top,df_tra,df_eval,tra_ds,eval_ds))

# raw,op,op_mid,op_big,op_top,df_tra,df_eval,tra_ds,eval_ds=pkl(path='raw.pkl',wbORrb='rb')

In [ ]:
from u import full2half
raw=pd.read_excel('診斷證明書_手術資料表(至0902登打)_v11.100.xlsx')
raw=raw.fillna('')

search=raw[['理賠案號','健保手術代碼']].values
raw['lb_name']=raw['理賠案號'].apply(lambda x:search[np.where(search[:,0]==x)][:,1].tolist())
print('before drop',len(raw))
s=['理賠案號','診斷-病名','醫囑','健保手術代碼']
raw=raw.drop_duplicates(subset=s).reset_index(drop=True)
print('aft drop',len(raw))
op=sorted(list(set(np.concatenate(raw['lb_name'].values).tolist())))
raw['lb']=raw['lb_name'].apply(lambda x:[0 if not i in op else op.index(i) for i in x])
raw['lb']=raw['lb'].apply(lambda x:[1 if i in x else 0 for i in range(len(op))])

op_mid=sorted(list(set([i[:4] for i in op])))
op_big=sorted(list(set([i[:2] for i in op])))
op_top=sorted(list(set([i[:1] for i in op])))

raw['lb_mid']=raw['lb_name'].apply(lambda x:[1 if op_mid[i][:4] in [j[:4] for j in x] else 0 for i in range(len(op_mid))])
raw['lb_big']=raw['lb_name'].apply(lambda x:[1 if op_big[i][:2] in [j[:2] for j in x] else 0 for i in range(len(op_big))])
raw['lb_top']=raw['lb_name'].apply(lambda x:[1 if op_top[i][:1] in [j[:1] for j in x] else 0 for i in range(len(op_top))])

raw['診斷-病名_']=raw['診斷-病名'].apply(lambda x:full2half(x).lower())
raw['醫囑_']=raw['醫囑'].apply(lambda x:full2half(x).lower())
pat1=['(空白)','〔空白〕','＜空白＞',
    '｛空白｝','【空白】','〈空白〉','《空白》','﹤空白﹥','﹛空白﹜','「空白」','［空白］','｢空白｣',
    '〔以下空白〕','(以下空白)','＜以下空白＞',
    '｛以下空白｝','【以下空白】','〈以下空白〉','《以下空白》','﹤以下空白﹥','﹛以下空白﹜',
    '「以下空白」','［以下空白］','｢以下空白｣'
    ]
pat2=['-{2,}空白-{2,}','\s+空白\s+','-{2,}以下空白-{2,}','-{2,}以   下   空   白-{2,}',
      '\s+以下空白\s+']
def rep(x):
#     print('0',x)
    for i in pat2:
        x=re.sub(i, '', x)
#     print('1',x)
    x=re.sub('['+'|'.join(pat1)+']', '', x)
#     print('2',x)
    x=re.sub('[以下空白]', '', x) 
    x=re.sub('[空白]', '', x) 
    x=re.sub('以下空白', '', x) 
#     print('3',x)
    return x
raw['診斷-病名_']=raw['診斷-病名_'].apply(lambda x:rep(x))
raw['醫囑_']=raw['醫囑_'].apply(lambda x:rep(x))
raw['醫囑__']=raw['醫囑_'].apply(lambda x:x.replace('。','[SEP]'))
raw['醫囑__']=raw['醫囑__'].apply(lambda x:re.sub('(\[SEP\]){2,}', '[SEP]', x) )
raw

In [ ]:
tok_path='ckip_albert_tok'#'tok_ckip_albert_tiny'
# mdl_path='mdl4seqCls_ckip_albert_tiny'
if os.path.exists(tok_path):
    tokenizer=BertTokenizerFast.from_pretrained(tok_path)
else:    
    tokenizer=BertTokenizerFast.from_pretrained('ckiplab/albert-tiny-chinese')
tokenizer

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='ckiplab/albert-tiny-chinese', vocab_size=21128, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
tra_r=.8
eval_r=.2
# test_r=.1
min_=3

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
def sep_df_more_less_multiLB(df,stratify_col_name,fraction):
    # n_splits多少個 cross validation sets
    msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=fraction, random_state=0)
    tra,test=None,None
    for tra_idx, test_idx in msss.split(df, np.array(df[stratify_col_name].values.tolist())):
        tra, test = df.iloc[tra_idx], df.iloc[test_idx]
    return tra,test

lb_num=np.concatenate(raw['lb_name'].values).tolist()
less=[i for i in op if lb_num.count(i)<=min_]
more=[i for i in op if lb_num.count(i)>min_]
raw['more_less']=raw['lb_name'].apply(lambda x:any([1 for i in x if i in more]))
(m,l)=(raw[raw['more_less']==1].reset_index(drop=True),raw[raw['more_less']==0].reset_index(drop=True))


# _,df_eval=sep_df_more_less_multiLB(m,'lb',eval_r/(tra_r+eval_r+test_r))
# df_tra,df_test=sep_df_more_less_multiLB(_,'lb',test_r/(tra_r+test_r))
# df_eval=pd.concat((df_eval,l)).reset_index(drop=True)
# df_tra=pd.concat((df_tra,l)).reset_index(drop=True)
# df_test=pd.concat((df_test,l)).reset_index(drop=True)
# len(df_eval),len(df_tra),len(df_test)

df_tra,df_eval=sep_df_more_less_multiLB(m,'lb',eval_r/(tra_r+eval_r))
len(df_eval),len(df_tra)

(708, 2870)

In [ ]:
# df_tra df_eval df_test
df_tra['inputs']=df_tra['醫囑__']+'[SEP]'+df_tra['診斷-病名_']
df_eval['inputs']=df_eval['醫囑__']+'[SEP]'+df_eval['診斷-病名_']
# df_test['inputs']=df_test['醫囑']+'[SEP]'+df_test['診斷-病名']

params={'return_token_type_ids': True,'return_tensors':'pt', 'padding':True, 'truncation':True,
    'return_attention_mask': True}
tra_inps=tokenizer.batch_encode_plus(df_tra['inputs'].values.tolist(),**params)
eval_inps=tokenizer.batch_encode_plus(df_eval['inputs'].values.tolist(), **params)
# test_inps=tokenizer.batch_encode_plus(df_test['inputs'].values.tolist(), **params)

p=[tra_inps['input_ids'],tra_inps['token_type_ids'],tra_inps['attention_mask'],torch.tensor(df_tra['lb'].tolist()).float()]
tra_ds = TensorDataset(*p)
p=[eval_inps['input_ids'],eval_inps['token_type_ids'],eval_inps['attention_mask'],torch.tensor(df_eval['lb'].tolist()).float()]
eval_ds = TensorDataset(*p)
# p=[test_inps['input_ids'],test_inps['token_type_ids'],test_inps['attention_mask'],torch.tensor(df_test['lb']).float()]
# test_ds = TensorDataset(*p)

/facecore/env/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/facecore/env/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# from torch.utils.data import DataLoader,TensorDataset,Dataset
# class CustomDataset(Dataset):
#     def __init__(self, inputs, df_):
#         self.inputs=inputs
#         self.df=df_
#     def __len__(self):
#         return len(self.df)
#     def __getitem__(self, idx):
#         return (self.inputs['input_ids'][idx],
#             self.inputs['token_type_ids'][idx], 
#             self.inputs['attention_mask'][idx],
#             torch.tensor([self.df['大分_l'].iloc[idx]]).float(),
#             torch.tensor([self.df['中分_l'].iloc[idx]]).float(),
#             torch.tensor([self.df['職業_l'].iloc[idx]]).float(),
#             torch.tensor(self.df['大分_i'].iloc[idx]).float(), #大分類
#             torch.tensor(self.df['中分_i'].iloc[idx]).float(), #職業中分類_富邦內部版
#             torch.tensor(self.df['職業_i'].iloc[idx]).float(), #職業代碼
#             torch.tensor(idx)) 
            ### return {'data': x, 'target': y}
# tra_ds=CustomDataset(tra_inps,df_tra)
# eval_ds=CustomDataset(eval_inps,df_eval)
# test_ds=CustomDataset(test_inps,df_test)

In [ ]:

# tokenizer.save_pretrained('ckip_albert_tok')
mdl.save_pretrained(dir_)

In [ ]:
# dir_='ckip_albert_mask'
# if os.path.exists(dir_):
#     mdl=AutoModel.from_pretrained(dir_) #ckip_bert_mask
# else:
mdl=AutoModel.from_pretrained('ckiplab/albert-base-chinese')

Downloading:   0%|          | 0.00/750 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Some weights of the model checkpoint at ckiplab/albert-base-chinese were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertModel were not initialized from the model checkpoint at ckiplab/albert-base-chinese and are newly initialized: ['albert.pooler.bias', 'albert.pooler.weight']
You should probably TRAIN this model on a down-stream t

In [ ]:
class PosModel(nn.Module):
    def __init__(self,mdl,lb_len):
        super(PosModel, self).__init__()
        self.base_model = mdl
        d=1/2
        emb=768
        self.dp = nn.Dropout(d)
        self.linear_clf = nn.Linear(emb, lb_len)
    def forward(self, inputs):
        outputs = self.base_model(inputs['input_ids'],
            token_type_ids=inputs['token_type_ids'],attention_mask=inputs['attention_mask'])
        out = self.dp(outputs.pooler_output)
        out = self.linear_clf(out)
        return outputs,out

model = PosModel(mdl,len(op))
model

In [ ]:
tra_dl=DataLoader(tra_ds,batch_size=2,shuffle=True)#,persistent_workers=False,num_workers=4
eval_dl=DataLoader(eval_ds,batch_size=512,shuffle=True,)#persistent_workers=False,num_workers=4
# test_dl=DataLoader(test_ds,batch_size=512,shuffle=True,persistent_workers=True,num_workers=4)#
len(tra_dl),len(eval_dl)#,len(test_dl)

(1435, 2)

In [ ]:
# df_tra df_eval df_test
# df_test['inputs']=df_test['醫囑']+'[SEP]'+df_test['診斷-病名']

raw['inputs']=raw['醫囑__']+'[SEP]'+raw['診斷-病名_']
params={'return_token_type_ids': True,'return_tensors':'pt', 'padding':True, 'truncation':True,
    'return_attention_mask': True}

inps=tokenizer.batch_encode_plus(raw['inputs'].values.tolist(),**params)
p=[inps['input_ids'],inps['token_type_ids'],inps['attention_mask'],torch.tensor(raw['lb'].tolist()).float()]
dl=DataLoader(TensorDataset(*p),batch_size=512,shuffle=True)#,persistent_workers=False,num_workers=4
# p=[test_inps['input_ids'],test_inps['token_type_ids'],test_inps['attention_mask'],torch.tensor(df_test['lb']).float()]
# test_ds = TensorDataset(*p)
len(dl)

7

In [ ]:
def train(dl):
    model.train()
    global optimizer
    train_loss=0
    for batch_idx, batch in tqdm(enumerate(dl),leave=False):
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'token_type_ids': batch[1],
                  'attention_mask': batch[2],
                 }
        optimizer.zero_grad()
        outputs,out = model(inputs)
        loss = criterion(out, batch[3])
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
#         print(loss.item())
#     print('train loss: %f' % (train_loss))
    return train_loss/(batch_idx+1)

def test(dl):
    model.eval()
#     trues_b,preds_b,trues_m,preds_m,trues_c,preds_c = (list(),list(),list(),list(),list(),list())
    trues = None
    preds = None
    eval_loss=0
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dl),leave=False):
            batch = tuple(b.to(device) for b in batch)
            inputs = {'input_ids':      batch[0],
                      'token_type_ids': batch[1],
                      'attention_mask': batch[2],
                     }
            outputs,out= model(inputs)
            loss = criterion(out, batch[3])
            eval_loss+=loss.item()
            if trues is None:
                trues=batch[3].cpu().numpy().copy()
                preds=torch.sigmoid(out).cpu().numpy().copy()
            else:
                trues=np.append(trues, np.array(batch[3].cpu().numpy()), axis=0)
                preds=np.append(preds, np.array(torch.sigmoid(out).cpu()), axis=0)
        f1=f1_score(trues,(preds>.5),average='micro')
        acc=accuracy_score(trues, (preds>.5))
    return acc, f1, eval_loss/(batch_idx+1)

def torch_save(params,path):
    torch.save(params,path)
def torch_load(path):
    return torch.load(path)
def is_save(i,tra_loss,eval_loss,acc,f1):
    global max_acc
    global tra_low_loss
    global eval_low_loss
    global eval_last_loss
    params={'epoch':i,'train_loss':tra_loss,'eval_loss':eval_loss,'acc':acc,'f1':f1}
    params['model_state_dict']=model.state_dict()
    params['optimizer_state_dict']=optimizer.state_dict()
    if max_acc<acc:
        torch_save(params,prefix+'_acc.'+ext)
        max_acc=acc
        print('max_acc!',i,f'tra:{tra_loss}',f'eval:{eval_loss}',f'acc:{acc}',f'f1:{f1}')
    if tra_loss<tra_low_loss:
        torch_save(params,prefix+'_tra.'+ext)
        tra_low_loss=tra_loss
        print('tra_loss!',i,f'tra:{tra_loss}',f'eval:{eval_loss}',f'acc:{acc}',f'f1:{f1}')
    if eval_loss<eval_low_loss:
        torch_save(params,prefix+'_eval.'+ext)
        eval_low_loss=eval_loss
        print('eval_loss!',i,f'tra:{tra_loss}',f'eval:{eval_loss}',f'acc:{acc}',f'f1:{f1}')
    if i%30==0:
        torch_save(params,prefix+f'_{i}.'+ext)
        print('30 epoch!',i,f'tra:{tra_loss}',f'eval:{eval_loss}',f'acc:{acc}',f'f1:{f1}')

In [ ]:
ckpt=torch_load('jc_hug_ckip_eval.pth')#jc_hug_ckip_last
# ckpt=torch.load('jc_hug_ckip_eval.pth',map_location='cpu')#jc_hug_ckip_eval
# model.load_state_dict(ckpt['model_state_dict'])
ckpt

In [ ]:
lr=1e-7 # 1e-6
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.999)
optimizer = optim.AdamW(model.parameters(), lr=lr,amsgrad=True,weight_decay=0.001)
# optimizer = torch_optimizer.Yogi(model.parameters(),lr=lr,weight_decay=1e-4)

In [ ]:
# criterion=nn.CrossEntropyLoss()
criterion=nn.BCEWithLogitsLoss()

# optimizer.load_state_dict(ckpt['optimizer_state_dict'])
model.to(device)
# ckpt

PosModel(
  (base_model): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(21128, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
       

In [ ]:
prefix='CL_hug_ckip'
ext='pth'
epoches=100000

# ckpt=torch_load('jc_hug_ckip_eval.pth')
# if ckpt:
#     max_b_acc=ckpt['acc_b']
#     max_m_acc=ckpt['acc_m']
#     max_c_acc=ckpt['acc_c']
#     tra_low_loss=ckpt['train_loss']
#     eval_low_loss=ckpt['eval_loss']
#     trigger_times=0
# else:
max_acc=0.2
tra_low_loss=0.02
eval_low_loss=0.02

In [ ]:
# st=ckpt['epoch']+1
st=1
for i in range(st,epoches+1):
    tra_loss=train(tra_dl)
    acc,f1,eval_loss=test(eval_dl)
    is_save(i,tra_loss,eval_loss,acc,f1)

0it [00:00, ?it/s]

In [ ]:
acc,f1,eval_loss=test(dl)

0it [00:00, ?it/s]

In [ ]:
acc,f1,eval_loss

(0.0, 0.02610503148499114, 0.6963042787143162)

In [ ]:
# a=torch.sigmoid(out).cpu().numpy()
# b=batch[3].cpu().numpy()
# f1_score(b,a,average='micro')
a.shape,b.shape

((207, 100), (207, 100))

In [ ]:

test_f1_accuracy = f1_score(b,(a>.5),average='micro')
test_flat_accuracy = accuracy_score(b, (a>.5))
test_f1_accuracy,test_flat_accuracy

(0.027357811375089996, 0.0)

In [ ]:
torch.sigmoid(out)

tensor([[0.5158, 0.4943, 0.4735,  ..., 0.4555, 0.5223, 0.5182],
        [0.5160, 0.4958, 0.4746,  ..., 0.4560, 0.5212, 0.5184],
        [0.5166, 0.4953, 0.4742,  ..., 0.4559, 0.5211, 0.5182],
        ...,
        [0.5100, 0.4963, 0.4749,  ..., 0.4623, 0.5165, 0.5173],
        [0.5145, 0.4966, 0.4744,  ..., 0.4566, 0.5210, 0.5185],
        [0.5144, 0.4962, 0.4745,  ..., 0.4564, 0.5215, 0.5182]],
       device='cuda:0')

In [ ]:
a[a > 0.5] = 1
a

tensor([[1.0000, 0.4943, 0.4735,  ..., 0.4555, 1.0000, 1.0000],
        [1.0000, 0.4958, 0.4746,  ..., 0.4560, 1.0000, 1.0000],
        [1.0000, 0.4953, 0.4742,  ..., 0.4559, 1.0000, 1.0000],
        ...,
        [1.0000, 0.4963, 0.4749,  ..., 0.4623, 1.0000, 1.0000],
        [1.0000, 0.4966, 0.4744,  ..., 0.4566, 1.0000, 1.0000],
        [1.0000, 0.4962, 0.4745,  ..., 0.4564, 1.0000, 1.0000]],
       device='cuda:0')

In [ ]:
criterion(out, batch[3])

tensor(0.6974, device='cuda:0')

In [ ]:
criterion(a, batch[3])

tensor(1.1391, device='cuda:0')